In [1]:
# !wget https://raw.githubusercontent.com/gururise/AlpacaDataCleaned/main/alpaca_data_cleaned.json

In [2]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
openai.api_type = 'azure'
openai.api_base = "https://husein-ai2-aiservices.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

engine = 'gpt-35-turbo'

In [4]:
def predict(prompt):
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
       
    ]
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=1024,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [5]:
with open('alpaca_data_cleaned.json','r') as fopen:
    all_objs = json.load(fopen)

In [6]:
from depth import createConstraintsPrompt, createDeepenPrompt, createConcretizingPrompt, createReasoningPrompt
from breadth import createBreadthPrompt

In [7]:
!mkdir output
# !rm output/*

mkdir: cannot create directory ‘output’: File exists


In [8]:
def generate(instruction, index):
    filename = f'output/{index}.json'
    if os.path.exists(filename):
        return
    
    try:
        evol_prompts = []
        evol_prompts.append(createConstraintsPrompt(instruction))
        evol_prompts.append(createDeepenPrompt(instruction))
        evol_prompts.append(createConcretizingPrompt(instruction))
        evol_prompts.append(createReasoningPrompt(instruction))
        evol_prompts.append(createBreadthPrompt(instruction))

        selected_evol_prompt = random.choice(evol_prompts)
        ins = predict(selected_evol_prompt)
        answer = predict(ins + ', jawab dalam bahasa melayu')
        d = {"instruction":ins,"output":answer}
        with open(filename, 'w') as fopen:
            json.dump(d, fopen)
    except:
        pass

In [9]:
instructions = []
for cur_obj in all_objs[:len(all_objs) // 2]:
    instructions.append(cur_obj['instruction'].strip() + '\r\n'+ cur_obj['input'].strip())

In [10]:
from tqdm import tqdm

max_worker = 3
for i in tqdm(range(0, len(instructions), max_worker)):
    b = instructions[i: i + max_worker]
    b = [(ins, i + no) for no, ins in enumerate(b)]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f[0], f[1]): f for f in b}

        for future in as_completed(futures):
            future.result()

 14%|█████▍                                | 1224/8627 [00:03<00:26, 282.98it/s]

'content'


 36%|█████████████▋                        | 3103/8627 [00:09<00:12, 435.05it/s]

'content'


 55%|█████████████████████                 | 4775/8627 [00:20<00:23, 167.24it/s]

...

 98%|████████████████████████████████████▎| 8471/8627 [1:55:37<14:45,  5.68s/it]

'content'


 99%|████████████████████████████████████▍| 8505/8627 [2:00:03<15:15,  7.51s/it]

'content'


100%|█████████████████████████████████████| 8627/8627 [2:20:24<00:00,  1.02it/s]
